In [5]:
import DataLoader
import importlib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import CoverageHelper
importlib.reload(DataLoader)

%matplotlib inline
test_graph_df = DataLoader.load_csv_test_graphs()
# stat_graph_df = DataLoader.load_csv_stat_graph("")
# stat_trans_graph_df = DataLoader.load_csv_stat_graph("-transitive")
stat_trans_act_graph_df = DataLoader.load_csv_stat_graph("-act-transitive")

Number of apps: 
376


Total entries:
39530


Total entries:
1949


Total entries:
39530


In [6]:
test_graph_df.head()


,source,target,action,widget_id,widget,strategy,app
0,com.learndrawing.howtodrawfnaf.MainActivity,NaN,NaN,NaN,NaN,systematic,com.learndrawing.howtodrawfnaf
1,com.learndrawing.howtodrawfnaf.MainActivity,NaN,NaN,NaN,NaN,random,com.learndrawing.howtodrawfnaf
2,com.sole.insider.free.SoleInsider,NaN,NaN,NaN,NaN,systematic,com.sole.insider.free
3,com.sole.insider.free.SoleInsider,NaN,NaN,NaN,NaN,random,com.sole.insider.free
4,com.lpp.MainActivity,com.unity3d.ads.android.view.UnityAdsFullscree...,back,NaN,NaN,systematic,com.CarsLiveWallpaperHQ


In [7]:
# Remove back?
# test_graph_df = test_graph_df[test_graph_df.action != "back"]

In [8]:
# stat_graph_df.head()

In [9]:
lol = stat_trans_act_graph_df[["app", "method", "widget"]]
lol[~lol.widget.notnull()].groupby(["app", "method"]).widget.value_counts(dropna=False).to_frame(
    name="widget_count").sort_values(by=["widget_count"], ascending=False)


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>widget_count</th>
    </tr>
    <tr>
      <th>app</th>
      <th>method</th>
      <th>widget</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>com.yinzcam.nfl.packers</th>
      <th>onCreate</th>
      <th>NaN</th>
      <td>1769</td>
    </tr>
    <tr>
      <th rowspan="4" valign="top">com.qxmd.readbyqxmd</th>
      <th>onResume</th>
      <th>NaN</th>
      <td>614</td>
    </tr>
    <tr>
      <th>onCreate</th>
      <th>NaN</th>
      <td>596</td>
    </tr>
    <tr>
      <th>onPause</th>
      <th>NaN</th>
      <td>527</td>
    </tr>
    <tr>
      <th>onActivityResult</th>
      <th>NaN</th>
      <td>485</td>
    </tr>
    <tr>
      <th>naukriApp.appModules.login</th>
      <th>c;android.intent.action.VIEW</th>
      <th>NaN</th>
      <td>485</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">com.qxmd.readbyqxmd</th>
      <th>onAttach</th>
      <th>NaN</th>
      <td>456</td>
    </tr>
    <tr>
      <th>onStart</th>
      <th>NaN</th>
      <td>453</td>
    </tr>
    <tr>
      <th>com.yinzcam.nfl.packers</th>
      <th>onActivityResult</th>
      <th>NaN</th>
      <td>333</td>
    </tr>
    <tr>
      <th>naukriApp.appModules.login</th>
      <th>onActivityResult</th>
      <th>NaN</th>
      <td>303</td>
    </tr>
    <tr>
      <th>com.newchic.client</th>
      <th>startActivityForResult</th>
      <th>NaN</th>
      <td>250</td>
    </tr>
    <tr>
      <th>com.yinzcam.nfl.packers</th>
      <th>handleLaunchFromPushNotification</th>
      <th>NaN</th>
      <td>247</td>
    </tr>
    <tr>
      <th>com.circle38.photocircle</th>
      <th>startActivityForResult</th>
      <th>NaN</th>
      <td>225</td>
    </tr>
    <tr>
      <th>com.yinzcam.nfl.packers</th>
      <th>onResume</th>
      <th>NaN</th>
      <td>222</td>
    </tr>
    <tr>
      <th>naukriApp.appModules.login</th>
      <th>onClick</th>
      <th>NaN</th>
      <td>207</td>
    </tr>
    <tr>
      <th>com.newchic.client</th>
      <th>startActivity</th>
      <th>NaN</th>
      <td>200</td>
    </tr>
    <tr>
      <th>naukriApp.appModules.login</th>
      <th>onViewCreated</th>
      <th>NaN</th>
      <td>190</td>
    </tr>
    <tr>
      <th>com.ifs.banking.fiid1603</th>
      <th>onCreate</th>
      <th>NaN</th>
      <td>184</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">naukriApp.appModules.login</th>
      <th>a</th>
      <th>NaN</th>
      <td>180</td>
    </tr>
    <tr>
      <th>setUserVisibleHint</th>
      <th>NaN</th>
      <td>143</td>
    </tr>
    <tr>
      <th>com.qxmd.readbyqxmd</th>
      <th>a</th>
      <th>NaN</th>
      <td>133</td>
    </tr>
    <tr>
      <th rowspan="5" valign="top">com.yinzcam.nfl.packers</th>
      <th>shouldOverrideUrlLoading;android.intent.action.VIEW</th>
      <th>NaN</th>
      <td>125</td>
    </tr>
    <tr>
      <th>onCreate;android.intent.action.MAIN</th>
      <th>NaN</th>
      <td>124</td>
    </tr>
    <tr>
      <th>checkLoginStatus</th>
      <th>NaN</th>
      <td>123</td>
    </tr>
    <tr>
      <th>promptUserLogin</th>
      <th>NaN</th>
      <td>122</td>
    </tr>
    <tr>
      <th>launchInfoPage</th>
      <th>NaN</th>
      <td>122</td>
    </tr>
    <tr>
      <th>com.ifs.banking.fiid1603</th>
      <th>s</th>
      <th>NaN</th>
      <td>120</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">com.thomsonreuters.reuterstv</th>
      <th>routeToActivity</th>
      <th>NaN</th>
      <td>116</td>
    </tr>
    <tr>
      <th>onCreate</th>
      <th>NaN</th>
      <td>115</td>
    </tr>
    <tr>
      <th>pt.worldit.joanmiro16</th>
      <th>onCreate</th>


In [10]:
# stat_trans_graph_df.head()

In [11]:
#difference
# pd.concat([stat_trans_graph_df,stat_trans_act_graph_df]).drop_duplicates(keep=False)


In [12]:
unique_widgets_per_app = test_graph_df.groupby(["strategy", "app"]).widget.value_counts(dropna=False).to_frame(
    name="widget_count")

unique_widgets_per_app.head()

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>widget_count</th>
    </tr>
    <tr>
      <th>strategy</th>
      <th>app</th>
      <th>widget</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="5" valign="top">random</th>
      <th rowspan="2" valign="top">alex.map.maps</th>
      <th>NaN</th>
      <td>3</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>2</td>
    </tr>
    <tr>
      <th>androidlab.carlog</th>
      <th>NaN</th>
      <td>1</td>
    </tr>
    <tr>
      <th>asp.aniemo.pack6.ext</th>
      <th>NaN</th>
      <td>1</td>
    </tr>
    <tr>
      <th>bestfreelivewallpapers.funny_photo_editor</th>
      <th>NaN</th>
      <td>1</td>
    </tr>
  </tbody>
</table>
</div>

In [13]:
unique_widgets_per_app['percentage'] = unique_widgets_per_app.groupby(level=0).apply(lambda x: 100 * x / x.sum())
unique_widgets_per_app

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>widget_count</th>
      <th>percentage</th>
    </tr>
    <tr>
      <th>strategy</th>
      <th>app</th>
      <th>widget</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="30" valign="top">random</th>
      <th rowspan="2" valign="top">alex.map.maps</th>
      <th>NaN</th>
      <td>3</td>
      <td>0.266430</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>2</td>
      <td>0.177620</td>
    </tr>
    <tr>
      <th>androidlab.carlog</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>asp.aniemo.pack6.ext</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>bestfreelivewallpapers.funny_photo_editor</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>bestfreelivewallpapers.love_photo_frames_hd</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>biz.andxor.hearingaid</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">br.com.blackmountain.mylook</th>
      <th>android.support.v7.widget.CardView</th>
      <td>2</td>
      <td>0.177620</td>
    </tr>
    <tr>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>br.com.geochenapps.tonguetwister</th>
      <th>android.widget.ImageButton</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>br.org.altitudesl</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">brdata.cms.base.foodbazaar</th>
      <th>android.widget.ImageSwitcher</th>
      <td>2</td>
      <td>0.177620</td>
    </tr>
    <tr>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">bth.studio.quickcharge</th>
      <th>android.widget.FrameLayout</th>
      <td>14</td>
      <td>1.243339</td>
    </tr>
    <tr>
      <th>android.widget.ToggleButton</th>
      <td>6</td>
      <td>0.532860</td>
    </tr>
    <tr>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">carrioncastillo.invitaciones.happy</th>
      <th>NaN</th>
      <td>2</td>
      <td>0.177620</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>2</td>
      <td>0.177620</td>
    </tr>
    <tr>
      <th>church.itvessel.com.tarm</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>co.appzone.www.eye_makeup</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>co.instavr.iostest</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>co.mobius.vrcinema</th>
      <th>android.widget.ImageButton</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>coloring.book.color</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>com.BillDirkes.QuickEM</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>com.CarsLiveWallpaperHQ</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th>com.CrochetPatternIdeas.BehMedia</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.088810</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">com.Feng.LingCrazyLockScreen</th>
      <th>android.widget.Button

In [14]:
number_of_unique_screens_per_app = test_graph_df[['strategy', 'app', 'source', 'target', 'widget']].melt(
    id_vars=['strategy', 'app', 'widget'])
number_of_unique_screens_per_app

,strategy,app,widget,variable,value
0,systematic,com.learndrawing.howtodrawfnaf,NaN,source,com.learndrawing.howtodrawfnaf.MainActivity
1,random,com.learndrawing.howtodrawfnaf,NaN,source,com.learndrawing.howtodrawfnaf.MainActivity
2,systematic,com.sole.insider.free,NaN,source,com.sole.insider.free.SoleInsider
3,random,com.sole.insider.free,NaN,source,com.sole.insider.free.SoleInsider
4,systematic,com.CarsLiveWallpaperHQ,NaN,source,com.lpp.MainActivity
5,random,com.CarsLiveWallpaperHQ,NaN,source,com.lpp.MainActivity
6,systematic,com.vadevteam.thandongdatviet,android.widget.Button,source,com.vadevteam.thandongdatviet.SplashScreen
7,systematic,com.vadevteam.thandongdatviet,NaN,source,com.vadevteam.thandongdatviet.SplashScreen
8,systematic,com.vadevteam.thandongdatviet,android.widget.Button,source,com.vadevteam.thandongdatviet.SplashScreen
9,systematic,com.vadevteam.thandongdatviet,NaN,source,com.vadevteam.thandongdatviet.SplashScreen


Edges from

In [15]:
systematic_test_graph = test_graph_df[test_graph_df.strategy == "systematic"]
systematic_test_graph.head()

,source,target,action,widget_id,widget,strategy,app
0,com.learndrawing.howtodrawfnaf.MainActivity,NaN,NaN,NaN,NaN,systematic,com.learndrawing.howtodrawfnaf
2,com.sole.insider.free.SoleInsider,NaN,NaN,NaN,NaN,systematic,com.sole.insider.free
4,com.lpp.MainActivity,com.unity3d.ads.android.view.UnityAdsFullscree...,back,NaN,NaN,systematic,com.CarsLiveWallpaperHQ
6,com.vadevteam.thandongdatviet.SplashScreen,com.vadevteam.thandongdatviet.ChoosenActivity,click,2.131296e+09,android.widget.Button,systematic,com.vadevteam.thandongdatviet
7,com.vadevteam.thandongdatviet.SplashScreen,com.vadevteam.thandongdatviet.ChoosenActivity,back,NaN,NaN,systematic,com.vadevteam.thandongdatviet


In [16]:
random_test_graph = test_graph_df[(test_graph_df.strategy == "random")]
random_test_graph.head()


,source,target,action,widget_id,widget,strategy,app
1,com.learndrawing.howtodrawfnaf.MainActivity,NaN,NaN,NaN,NaN,random,com.learndrawing.howtodrawfnaf
3,com.sole.insider.free.SoleInsider,NaN,NaN,NaN,NaN,random,com.sole.insider.free
5,com.lpp.MainActivity,com.unity3d.ads.android.view.UnityAdsFullscree...,back,NaN,NaN,random,com.CarsLiveWallpaperHQ
15,com.vadevteam.thandongdatviet.DetailActivity,com.vadevteam.thandongdatviet.MainActivity,back,NaN,NaN,random,com.vadevteam.thandongdatviet
16,com.vadevteam.thandongdatviet.SplashScreen,com.vadevteam.thandongdatviet.DetailActivity,click,2.131296e+09,android.widget.Button,random,com.vadevteam.thandongdatviet


Systematic:

In [17]:
direct_edges_not_found = CoverageHelper.diff_screens(stat_trans_act_graph_df, systematic_test_graph)
direct_edges_not_found.head()

,package,source,target,method,widget,app
0,com.vadevteam.thandongdatviet,com.vadevteam.thandongdatviet.MainActivity,com.vadevteam.thandongdatviet.DetailActivity,startImagePagerActivity,android.widget.ListView,com.vadevteam.thandongdatviet
1,com.vadevteam.thandongdatviet,com.vadevteam.thandongdatviet.ChoosenActivity,com.vadevteam.thandongdatviet.MainActivity,onClick,android.widget.Button,com.vadevteam.thandongdatviet
2,com.vadevteam.thandongdatviet,com.vadevteam.thandongdatviet.ChoosenActivity,com.vadevteam.thandongdatviet.DetailActivity,onClick,android.widget.Button,com.vadevteam.thandongdatviet
3,com.yinzcam.nfl.packers,com.yinzcam.nba.mobile.settings.application.Ap...,com.yinzcam.nba.mobile.gamestats.recap.RecapAc...,onClick,android.widget.ImageView,com.yinzcam.nfl.packers
4,com.yinzcam.nfl.packers,com.yinzcam.nba.mobile.settings.application.Ap...,com.yinzcam.nba.mobile.gamestats.recap.RecapAc...,onClick,android.widget.ListView,com.yinzcam.nfl.packers


widgets from missed edges from systematic test graphs

In [18]:
# unique_widgets_per_app = CoverageHelper.widget_counter(direct_edges_not_found)
# unique_widgets_per_app.head()

unique_widgets_per_app = direct_edges_not_found.groupby("method").widget.value_counts(dropna=False).to_frame(
    name="widget_count")
unique_widgets_per_app

,,widget_count
method,widget,
A,NaN,1
H,NaN,1
Historico,android.widget.Button,1
I,NaN,1
IniciaApp,NaN,1
J,android.app.AlertDialog$Builder,2
K,NaN,1
L,android.app.AlertDialog$Builder,1
MenuItemSelected,NaN,4


In [19]:
unique_widgets_per_app['percentage'] = unique_widgets_per_app.apply(lambda x: 100 * x / x.sum())
unique_widgets_per_app.sort_values(by=["widget_count"], ascending=False)

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>widget_count</th>
      <th>percentage</th>
    </tr>
    <tr>
      <th>method</th>
      <th>widget</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>onCreate</th>
      <th>NaN</th>
      <td>278</td>
      <td>8.259061</td>
    </tr>
    <tr>
      <th>onOptionsItemSelected</th>
      <th>android.view.MenuItem</th>
      <td>165</td>
      <td>4.901961</td>
    </tr>
    <tr>
      <th>onClick</th>
      <th>android.widget.ImageView</th>
      <td>128</td>
      <td>3.802733</td>
    </tr>
    <tr>
      <th>onActivityResult</th>
      <th>NaN</th>
      <td>115</td>
      <td>3.416518</td>
    </tr>
    <tr>
      <th>onResume</th>
      <th>NaN</th>
      <td>110</td>
      <td>3.267974</td>
    </tr>
    <tr>
      <th>onClick</th>
      <th>android.widget.Button</th>
      <td>85</td>
      <td>2.525253</td>
    </tr>
    <tr>
      <th>a</th>
      <th>NaN</th>
      <td>75</td>
      <td>2.228164</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">onClick</th>
      <th>android.view.View</th>
      <td>70</td>
      <td>2.079620</td>
    </tr>
    <tr>
      <th>android.widget.ListView</th>
      <td>57</td>
      <td>1.693405</td>
    </tr>
    <tr>
      <th>onItemClick</th>
      <th>android.view.View</th>
      <td>57</td>
      <td>1.693405</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">onClick</th>
      <th>android.widget.RelativeLayout</th>
      <td>54</td>
      <td>1.604278</td>
    </tr>
    <tr>
      <th>android.app.AlertDialog$Builder</th>
      <td>52</td>
      <td>1.544860</td>
    </tr>
    <tr>
      <th>onMenuItemClick</th>
      <th>android.widget.PopupMenu</th>
      <td>49</td>
      <td>1.455734</td>
    </tr>
    <tr>
      <th>onItemClick</th>
      <th>android.widget.ListView</th>
      <td>44</td>
      <td>1.307190</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">onCreateView</th>
      <th>android.view.ViewGroup</th>
      <td>44</td>
      <td>1.307190</td>
    </tr>
    <tr>
      <th>android.view.LayoutInflater</th>
      <td>44</td>
      <td>1.307190</td>
    </tr>
    <tr>
      <th>onStart</th>
      <th>NaN</th>
      <td>39</td>
      <td>1.158645</td>
    </tr>
    <tr>
      <th>startActivityForResult</th>
      <th>NaN</th>
      <td>35</td>
      <td>1.039810</td>
    </tr>
    <tr>
      <th>onAttach</th>
      <th>NaN</th>
      <td>35</td>
      <td>1.039810</td>
    </tr>
    <tr>
      <th>onPause</th>
      <th>NaN</th>
      <td>33</td>
      <td>0.980392</td>
    </tr>
    <tr>
      <th>onNewIntent</th>
      <th>NaN</th>
      <td>32</td>
      <td>0.950683</td>
    </tr>
    <tr>
      <th>onClick</th>
      <th>android.widget.TextView</th>
      <td>29</td>
      <td>0.861557</td>
    </tr>
    <tr>
      <th>a</th>
      <th>android.widget.ListView</th>
      <td>26</td>
      <td>0.772430</td>
    </tr>
    <tr>
      <th rowspan="6" valign="top">onEditorAction</th>
      <th>com.qxmd.readbyqxmd.activities.ch</th>
      <td>25</td>
      <td>0.742721</td>
    </tr>
    <tr>
      <th>com.qxmd.readbyqxmd.activities.cn</th>
      <td>25</td>
      <td>0.742721</td>
    </tr>
    <tr>
      <th>com.qxmd.readbyqxmd.activities.fm</th>
      <td>25</td>
      <td>0.742721</td>
    </tr>
    <tr>
      <th>com.qxmd.readbyqxmd.activities.fn</th>
      <td>25</td>
      <td>0.742721</td>
    </tr>
    <tr>
      <th>com.qxmd.readbyqxmd.activities.cj</th>
      <td>25</td>
      <td>0.742721</td>
    </tr>
    <tr>
      <th>android.widget.EditText</th>
      <td>25</td>
      <td>0.742721</td>
    </tr>
    <tr>
      <th>onClick</th>
      <th>androi

Random:

In [20]:
direct_edges_not_found = CoverageHelper.diff_screens(stat_trans_act_graph_df, random_test_graph)
direct_edges_not_found.head()

,package,source,target,method,widget,app
0,com.vadevteam.thandongdatviet,com.vadevteam.thandongdatviet.ChoosenActivity,com.vadevteam.thandongdatviet.DetailActivity,onClick,android.widget.Button,com.vadevteam.thandongdatviet
1,com.yinzcam.nfl.packers,com.yinzcam.nba.mobile.home.NBAHomeActivity,com.yinzcam.nba.mobile.bracket.BracketActivity,onNewIntent,NaN,com.yinzcam.nfl.packers
2,com.yinzcam.nfl.packers,com.yinzcam.nba.mobile.home.NBAHomeActivity,com.yinzcam.nba.mobile.live.highlights.Highlig...,onCreate,NaN,com.yinzcam.nfl.packers
3,com.yinzcam.nfl.packers,com.yinzcam.nba.mobile.home.NBAHomeActivity,com.yinzcam.nba.mobile.gamestats.recap.RecapAc...,onCreate,NaN,com.yinzcam.nfl.packers
4,com.yinzcam.nfl.packers,com.yinzcam.nba.mobile.home.NBAHomeActivity,com.yinzcam.nba.mobile.settings.application.Ap...,run,com.yinzcam.common.android.util.Popup$ForcedAl...,com.yinzcam.nfl.packers


In [21]:
unique_widgets_per_app = CoverageHelper.widget_counter(direct_edges_not_found)
unique_widgets_per_app.sort_values(by=["widget_count"], ascending=False)

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>widget_count</th>
      <th>percentage</th>
    </tr>
    <tr>
      <th>method</th>
      <th>widget</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>onCreate</th>
      <th>NaN</th>
      <td>286</td>
      <td>7.084469</td>
    </tr>
    <tr>
      <th>onOptionsItemSelected</th>
      <th>android.view.MenuItem</th>
      <td>175</td>
      <td>4.334902</td>
    </tr>
    <tr>
      <th>onClick</th>
      <th>android.widget.ImageView</th>
      <td>129</td>
      <td>3.195442</td>
    </tr>
    <tr>
      <th>onActivityResult</th>
      <th>NaN</th>
      <td>122</td>
      <td>3.022046</td>
    </tr>
    <tr>
      <th>onResume</th>
      <th>NaN</th>
      <td>109</td>
      <td>2.700025</td>
    </tr>
    <tr>
      <th>onClick</th>
      <th>android.widget.Button</th>
      <td>89</td>
      <td>2.204607</td>
    </tr>
    <tr>
      <th>a</th>
      <th>NaN</th>
      <td>74</td>
      <td>1.833044</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">onClick</th>
      <th>android.view.View</th>
      <td>65</td>
      <td>1.610107</td>
    </tr>
    <tr>
      <th>android.widget.RelativeLayout</th>
      <td>64</td>
      <td>1.585336</td>
    </tr>
    <tr>
      <th>a</th>
      <th>android.widget.ListView</th>
      <td>55</td>
      <td>1.362398</td>
    </tr>
    <tr>
      <th>onItemClick</th>
      <th>android.view.View</th>
      <td>55</td>
      <td>1.362398</td>
    </tr>
    <tr>
      <th>onClick</th>
      <th>android.app.AlertDialog$Builder</th>
      <td>54</td>
      <td>1.337627</td>
    </tr>
    <tr>
      <th>startActivityForResult</th>
      <th>NaN</th>
      <td>54</td>
      <td>1.337627</td>
    </tr>
    <tr>
      <th>onMenuItemClick</th>
      <th>android.widget.PopupMenu</th>
      <td>49</td>
      <td>1.213773</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">onCreateView</th>
      <th>android.view.LayoutInflater</th>
      <td>44</td>
      <td>1.089918</td>
    </tr>
    <tr>
      <th>android.view.ViewGroup</th>
      <td>44</td>
      <td>1.089918</td>
    </tr>
    <tr>
      <th>onItemClick</th>
      <th>android.widget.ListView</th>
      <td>44</td>
      <td>1.089918</td>
    </tr>
    <tr>
      <th>onClick</th>
      <th>android.widget.ListView</th>
      <td>41</td>
      <td>1.015606</td>
    </tr>
    <tr>
      <th>onStart</th>
      <th>NaN</th>
      <td>38</td>
      <td>0.941293</td>
    </tr>
    <tr>
      <th>onNewIntent</th>
      <th>NaN</th>
      <td>36</td>
      <td>0.891751</td>
    </tr>
    <tr>
      <th>onAttach</th>
      <th>NaN</th>
      <td>35</td>
      <td>0.866980</td>
    </tr>
    <tr>
      <th>onPause</th>
      <th>NaN</th>
      <td>33</td>
      <td>0.817439</td>
    </tr>
    <tr>
      <th>onClick</th>
      <th>android.widget.TextView</th>
      <td>30</td>
      <td>0.743126</td>
    </tr>
    <tr>
      <th rowspan="6" valign="top">onEditorAction</th>
      <th>com.qxmd.readbyqxmd.activities.fn</th>
      <td>25</td>
      <td>0.619272</td>
    </tr>
    <tr>
      <th>android.widget.EditText</th>
      <td>25</td>
      <td>0.619272</td>
    </tr>
    <tr>
      <th>com.qxmd.readbyqxmd.activities.fm</th>
      <td>25</td>
      <td>0.619272</td>
    </tr>
    <tr>
      <th>com.qxmd.readbyqxmd.activities.cn</th>
      <td>25</td>
      <td>0.619272</td>
    </tr>
    <tr>
      <th>com.qxmd.readbyqxmd.activities.cj</th>
      <td>25</td>
      <td>0.619272</td>
    </tr>
    <tr>
      <th>com.qxmd.readbyqxmd.activities.ch</th>
      <td>25</td>
      <td>0.619272</td>
    </tr>
    <tr>
      <th>onClick</th>
      <th>android.

In [22]:
unique_widgets_per_app.count()

widget_count    1226
percentage      1226
dtype: int64